<a href="https://colab.research.google.com/github/dhmodi/datascience/blob/master/Zomato_Rating_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
### To Upload data file
from google.colab import files
uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving Eatery_TrainingData.csv to Eatery_TrainingData.csv
User uploaded file "Eatery_TrainingData.csv" with length 11694253 bytes


# New Section

In [0]:
import pandas as pd
import io
import numpy
df = pd.read_csv('Eatery_TrainingData.csv')
testDF = pd.read_csv('Eatery_TestData.csv')

In [0]:
#print(df['featured_image'])

def f(row):
	if not row['featured_image']:
		val = 0
	else:
		val = 1
	return val

df['has_featured_image'] = df.apply(f, axis=1)
testDF['has_featured_image'] = testDF.apply(f, axis=1)


#print(df[['id','has_featured_image']])


In [51]:
enhancedDF = df.drop(['average_cost_for_two','cuisines','offers','zomato_events','user_rating__rating_tool_tip','book_url','currency','establishment_types__establishment_type__id','name','thumb','featured_image','location__address','location__city','location__city_id','location__country_id','location__latitude','location__locality','location__locality_verbose','location__longitude','location__zipcode','user_rating__custom_rating_text_background','user_rating__custom_rating_text'],axis=1)
#print(enhancedDF)
dummiesDF = pd.get_dummies(enhancedDF)


enhancedDF = testDF.drop(['average_cost_for_two','cuisines','offers','zomato_events','user_rating__rating_tool_tip','book_url','currency','establishment_types__establishment_type__id','name','thumb','featured_image','location__address','location__city','location__city_id','location__country_id','location__latitude','location__locality','location__locality_verbose','location__longitude','location__zipcode','user_rating__custom_rating_text_background','user_rating__custom_rating_text'],axis=1)
#print(enhancedDF)
dummiesTestDF = pd.get_dummies(enhancedDF)

cols = list(dummiesDF.columns.values) #Make a list of all of the columns in the df
cols.pop(cols.index('Zomato_user_rating')) #Remove b from list
dummiesDF = dummiesDF[cols+['Zomato_user_rating']] #Create new dataframe with columns in the order you want

cols = list(dummiesTestDF.columns.values) #Make a list of all of the columns in the df
cols.pop(cols.index('Zomato_user_rating')) #Remove b from list
dummiesTestDF = dummiesTestDF[cols+['Zomato_user_rating']] #Create new dataframe with columns in the order you want

print(dummiesDF.columns)
print(dummiesTestDF.columns)


Index(['id', 'has_online_delivery', 'has_table_booking', 'include_bogo_offers',
       'is_book_form_web_view', 'is_delivering_now',
       'is_table_reservation_supported', 'is_zomato_book_res',
       'medio_provider', 'opentable_support', 'price_range',
       'switch_to_order_menu', 'user_rating__votes', 'has_featured_image',
       'establishment_types__establishment_type__name_Bakery',
       'establishment_types__establishment_type__name_Bar',
       'establishment_types__establishment_type__name_Beer Garden',
       'establishment_types__establishment_type__name_Beverage Shop',
       'establishment_types__establishment_type__name_Bistro',
       'establishment_types__establishment_type__name_Café',
       'establishment_types__establishment_type__name_Casual Dining',
       'establishment_types__establishment_type__name_Club',
       'establishment_types__establishment_type__name_Cocktail Bar',
       'establishment_types__establishment_type__name_Dessert Parlour',
       'est

In [5]:
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

Using TensorFlow backend.


In [43]:
dummiesDF = dummiesDF.dropna()
#print(dummiesDF)
dummiesDF.set_index('id')
dataset = dummiesDF.values

# split into input (X) and output (Y) variables
X = dataset[:,1:81]
Y = dataset[:,81]

print(X)
print(Y)

#dummiesTestDF = dummiesDF.dropna()
#print(dummiesDF)
dummiesTestDF.set_index('id')
dataset = dummiesTestDF.values

# split into input (X) and output (Y) variables
X_test = dataset[:,1:81]

print(X_test)




[[0 0 True ... 0 0 0]
 [0 1 True ... 0 0 0]
 [0 1 True ... 0 0 0]
 ...
 [0 0 True ... 0 0 0]
 [1 1 True ... 0 0 0]
 [0 1 True ... 0 0 0]]
[3.6 4.4 4.1 4.9 4.5 4.0 3.5 4.1 4.3 3.9 4.7 4.3 4.7 3.6 4.6 3.3 4.4 4.5
 2.7 4.1 3.4 4.0 4.6 4.0 4.2 4.3 4.0 4.0 4.2 3.9 4.3 4.2 4.0 4.9 3.9 4.5
 4.3 4.3 3.8 4.5 4.3 4.1 3.9 4.1 4.1 4.4 4.2 4.0 4.1 3.7 4.1 4.4 4.2 3.8
 4.1 3.9 3.5 4.0 4.9 3.2 3.8 4.2 4.1 3.6 4.4 4.5 4.1 4.1 4.2 3.8 3.7 4.0
 4.4 4.3 3.8 3.8 3.8 4.1 4.1 4.3 3.4 4.4 4.1 4.8 3.9 4.2 3.9 4.5 4.2 3.4
 4.7 4.1 3.9 4.0 4.9 3.9 4.5 4.3 4.2 4.3 3.9 4.1 3.6 3.6 4.7 4.0 4.3 4.5
 4.0 3.7 4.0 4.6 4.3 4.1 4.7 3.6 4.3 4.1 3.8 3.2 4.0 4.4 4.2 4.0 3.8 4.1
 4.1 4.3 4.0 4.5 4.4 4.0 4.2 4.3 4.0 3.9 4.4 3.9 3.2 4.0 2.9 3.7 4.4 3.6
 4.3 3.9 3.7 3.9 4.5 3.6 4.5 4.1 3.7 4.5 3.7 4.2 3.9 4.0 4.3 3.6 4.1 4.3
 4.1 4.3 4.2 4.5 4.2 4.5 4.6 4.0 4.8 4.1 3.6 4.3 3.8 4.7 3.8 4.0 3.8 4.7
 3.8 4.3 4.6 4.0 4.0 4.5 3.3 3.7 4.6 4.2 3.8 3.8 4.1 4.4 3.6 4.3 4.3 4.2
 3.6 3.7 4.3 4.0 4.4 3.8 4.2 3.9 2.5 4.0 4.5 4.2 4.4 4.1 4.

In [0]:
# define base model
def baseline_model():
	# create model
	model = Sequential()
	model.add(Dense(80, input_dim=80, kernel_initializer='normal', activation='relu'))
	model.add(Dense(1, kernel_initializer='normal'))
	# Compile model
	model.compile(loss='mean_squared_error', optimizer='adam')
	return model

In [0]:
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
# evaluate model with standardized dataset
estimator = KerasRegressor(build_fn=baseline_model, epochs=100, batch_size=500, verbose=0)

In [46]:
kfold = KFold(n_splits=2, random_state=seed)
results = cross_val_score(estimator, X, Y, cv=kfold)
print("Results: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Results: -1.38 (1.10) MSE


In [49]:
estimator.fit(X,Y)
prediction = estimator.predict(X_test)

print (Y)
print (prediction)

ValueError: ignored

In [30]:
# evaluate model with standardized dataset
numpy.random.seed(seed)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=baseline_model, epochs=50, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(pipeline, X, Y, cv=kfold)
print("Standardized: %.2f (%.2f) MSE" % (results.mean(), results.std()))

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by 

Standardized: -0.03 (0.01) MSE


In [0]:
# define the model
def larger_model():
  # create model
	model = Sequential()
	model.add(Dense(80, input_dim=80, kernel_initializer='normal', activation='relu'))
	model.add(Dense(30, kernel_initializer='normal', activation='relu'))
	model.add(Dense(10, kernel_initializer='normal', activation='relu'))
	model.add(Dense(1, kernel_initializer='normal'))
	# Compile model
	model.compile(loss='mean_squared_error', optimizer='adam')
	return model

In [34]:
numpy.random.seed(seed)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=larger_model, epochs=50, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(pipeline, X, Y, cv=kfold)
print("Larger: %.2f (%.2f) MSE" % (results.mean(), results.std()))

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by 

Larger: -0.04 (0.01) MSE


In [0]:
# define wider model
def wider_model():
	# create model
	model = Sequential()
	model.add(Dense(100, input_dim=80, kernel_initializer='normal', activation='relu'))
	model.add(Dense(1, kernel_initializer='normal'))
	# Compile model
	model.compile(loss='mean_squared_error', optimizer='adam')
	return model

In [36]:
numpy.random.seed(seed)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=wider_model, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(pipeline, X, Y, cv=kfold)
print("Wider: %.2f (%.2f) MSE" % (results.mean(), results.std()))

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by 

Wider: -0.04 (0.02) MSE
